# Cell DT Platform - Python Interface

Демонстрация использования платформы из Jupyter Notebook

In [ ]:
import cell_dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-v0_8')

## 1. Создание симуляции

In [ ]:
# Создаем симуляцию с 100 клетками
sim = cell_dt.PySimulation(
    max_steps=500,
    dt=0.1,
    num_threads=4,
    seed=42
)

sim.create_population_with_transcriptome(100)
print(f"Created {sim.cell_count()} cells")

## 2. Регистрация модулей

In [ ]:
# Настраиваем параметры клеточного цикла
params = cell_dt.PyCellCycleParams(
    base_cycle_time=24.0,
    checkpoint_strictness=0.2,
    nutrient_availability=0.95,
    growth_factor_level=0.9,
    random_variation=0.3
)

sim.register_modules(
    enable_centriole=True,
    enable_cell_cycle=True,
    enable_transcriptome=True,
    cell_cycle_params=params
)

## 3. Запуск симуляции

In [ ]:
# Запускаем 200 шагов
cells = sim.step(200)
print(f"Step {sim.current_step()}: {len(cells)} cells")

## 4. Анализ данных

In [ ]:
# Преобразуем в pandas DataFrame
data = []
for cell in cells:
    data.append({
        'cell_id': cell.cell_id,
        'phase': cell.cell_cycle.phase,
        'cycle_progress': cell.cell_cycle.progress,
        'cycle_count': cell.cell_cycle.cycle_count,
        'mother_maturity': cell.centriole.mother_maturity,
        'daughter_maturity': cell.centriole.daughter_maturity,
        'mtoc_activity': cell.centriole.mtoc_activity,
        'cilium_present': cell.centriole.cilium_present,
    })

df = pd.DataFrame(data)
df.head()

In [ ]:
# Статистика по фазам
phase_counts = df['phase'].value_counts()
print("Phase distribution:")
print(phase_counts)

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Распределение фаз
phase_counts.plot(kind='bar', ax=axes[0, 0], color=['blue', 'green', 'orange', 'red'])
axes[0, 0].set_title('Phase Distribution')
axes[0, 0].set_xlabel('Phase')
axes[0, 0].set_ylabel('Count')

# Зрелость центриолей
axes[0, 1].hist(df['mother_maturity'], bins=20, alpha=0.5, label='Mother')
axes[0, 1].hist(df['daughter_maturity'], bins=20, alpha=0.5, label='Daughter')
axes[0, 1].set_title('Centriole Maturity')
axes[0, 1].set_xlabel('Maturity')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend()

# MTOC Activity
axes[1, 0].hist(df['mtoc_activity'], bins=20, color='green', alpha=0.7)
axes[1, 0].set_title('MTOC Activity')
axes[1, 0].set_xlabel('Activity')
axes[1, 0].set_ylabel('Count')

# Cycle Progress
for phase in ['G1', 'S', 'G2', 'M']:
    phase_data = df[df['phase'] == phase]['cycle_progress']
    if len(phase_data) > 0:
        axes[1, 1].hist(phase_data, bins=10, alpha=0.5, label=phase)
axes[1, 1].set_title('Cycle Progress by Phase')
axes[1, 1].set_xlabel('Progress')
axes[1, 1].set_ylabel('Count')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 5. Машинное обучение на данных симуляции

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Подготовка данных для ML
X = df[['mother_maturity', 'daughter_maturity', 'mtoc_activity', 'cycle_progress']]
y = df['phase']

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучение модели
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Оценка
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Важность признаков
importance = pd.DataFrame({
    'feature': X.columns,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature importance:")
print(importance)